<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/annelie/notebooks/Huggingface_QA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2: Evaluate Dataset

In [2]:
!pip install word2number

import json
import time
import google.generativeai as genai
from google.colab import userdata
import requests
from word2number import w2n
import re
import pandas as pd

# Gemini API Setup
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# Read dataset file
url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
data = pd.read_json(url)

data.head()

,type,question,options,intended_answer,context,timestamp
0,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Absolutely, I'm totally okay with data process...",2024-12-26 13:54:58.593
1,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yeah, you have my full consent for data proces...",2024-12-26 13:54:58.593
2,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Oh, data processing consent? Yes, you can go r...",2024-12-26 13:54:58.593
3,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,Regarding your request for data processing con...,2024-12-26 13:54:58.593
4,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yes, please proceed with the data processing, ...",2024-12-26 13:54:58.593


In [3]:
def convert_numbers_in_text(text):
    # Regular expression to find number words contained in questionnaires
    pattern = r'(two thousand|two hundred one|two hundred|fifty-one|thirty-one|twenty-one|sixteen|fifteen|eleven|thirty|twenty|fifty|forty|sixty|ten|five|six|one)'
    # Interesting finding: Regex only works if longer words are in order before shorter that contain similar parts, e.g. fifty-one has to be in front of fifty to work as intended

    def convert(match):
        word = match.group(0)
        try:
            # Convert the word to number
            return str(w2n.word_to_num(word))
        except ValueError:
            return word

    # Replace all number words in the text with their integer equivalents
    converted_text = re.sub(pattern, convert, text, flags=re.IGNORECASE)

    # Now convert ranges like 'twenty to thirty' into '20-30'
    converted_text = re.sub(r'(\d+)\s*(to|and)\s*(\d+)', r'\1-\3', converted_text)

    # Replace text
    # Todo: Dafür noch ne bessere Lösung finden, das ist eig nur n Beispiel und geht auch bei ähnlichen Sätzen nicht
    converted_text = converted_text.replace('more than 2000', 'larger than 2000')
    converted_text = converted_text.replace('More than 2000', 'Larger than 2000')

    return converted_text


def is_exact_or_phrase_match(option, text):
    # Escape the option to handle special characters
    escaped_option = re.escape(option.strip())

    # Pattern to match the option as a full word or part of a phrase
    pattern = rf'\b(?:\w+\s+)*{escaped_option}(?:\s+\w+)*\b'

    # Search for the pattern in the text (case-insensitive)
    return re.search(pattern, text, re.IGNORECASE) is not None

In [3]:
from transformers import pipeline

qa_pipeline1 = pipeline("question-answering", model="deepset/roberta-base-squad2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
qa_pipeline2 = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

In [ ]:
qa_pipeline3 = pipeline("question-answering", model='google-bert/bert-large-uncased-whole-word-masking-finetuned-squad')

## Evaluate Dataset with Pre-trained Models

In [5]:
from pickle import NONE
!pip install dateparser
import dateparser
from datetime import datetime
from datetime import timedelta

def predict_answers(df, pipeline):
    """
    Predict the answer for each row in the DataFrame.
    Prints only incorrectly predicted answers.
    """
    print("[INFO] Printing only incorrectly predicted answers.")
    correct_count = 0
    total_count = 0
    qa_pipeline = pipeline

    for _, row in df.iterrows():

        predictions = []

        # Regex-check only for single- and multi-select questions
        if row['options']:  # Evaluates to False if options is None or empty
            converted_context = convert_numbers_in_text(row['context'])

            for option in row['options']:
                # Check for exact match or part of a phrase
                exact_match = is_exact_or_phrase_match(option, converted_context)
                if exact_match:
                    predictions.append((option, 0.95))  # 95% confidence for exact match
                else:
                    result = qa_pipeline(question=row['question'], context=f"{converted_context} {option}")
                    predictions.append((option, result['score']))

        is_correct = False

        # Handle different question types
        if row['type'] == "SINGLE_SELECT":
            # Predict the option with the highest confidence
            predicted_option, confidence = max(predictions, key=lambda x: x[1])
            is_correct = predicted_option == row['intended_answer']

        elif row['type'] == "MULTI_SELECT":
            predicted_option = [option for option, score in predictions if score >= 0.95]
            is_correct = set(predicted_option) == set(row['intended_answer'])

        elif row['type'] == "DATE":
            try:
                # Basis-Timestamp aus der Dataframe-Spalte (Unix-Timestamp)
                base_timestamp = pd.Timestamp(row['timestamp'], unit='ms')

                # Extrahiere Zeitangabe aus dem Kontext
                extracted_time = qa_pipeline(question=row['question'], context=row['context'])['answer']

                # Konvertiere extrahierte Zeitangabe in Sekunden
                parsed_seconds = dateparser.parse(
                    extracted_time,
                    settings={'RELATIVE_BASE': base_timestamp.to_pydatetime()}
                )
                if not parsed_seconds:
                    raise ValueError(f"Unable to parse date from extracted time: {extracted_time}")

                predicted_option = parsed_seconds
                intended_seconds = int(row['intended_answer'])
                intended_date = base_timestamp + timedelta(seconds=intended_seconds)

                # Vergleich der vorhergesagten und intendierten Daten
                is_correct = predicted_option.date() == intended_date.date()

            except Exception as e:
                print(f"[ERROR] DATE question processing failed: {e}")

        elif row['type'] == "NUMBER":
            try:
                predicted_option = qa_pipeline(question=row['question'], context=row['context'])['answer']
                is_correct = predicted_option == row['intended_answer']
            except Exception as e:
                print(f"[ERROR] NUMBER question failed: {e}")

        elif row['type'] == "TEXT":
            continue
            # predicted_option = row['context']
            # is_correct = True

        # Output incorrect predictions
        if not is_correct:
            print(f"Context: {row['context']}")
            print(f"Correct: {row['intended_answer']}, Predicted: {predicted_option}")
            print()

        if is_correct:
            correct_count += 1
        total_count += 1

    # Calculate accuracy
    accuracy = correct_count / total_count if total_count > 0 else 0
    return accuracy


In [62]:
accuracy = predict_answers(data, qa_pipeline1)
print(f"Accuracy: {accuracy * 100:.2f} %")

[INFO] Printing only incorrectly predicted answers.
Context: Okay, so when you're talking about the customer group, we're definitely thinking about folks who are either a wholesaler or a distributor, that’s who we're focused on here.
Correct: Wholesaler, Distributor, Predicted: End User

Context: Well, the customer group we need to consider in this case, is made up of both the wholesaler and the distributor types, you see.
Correct: Wholesaler, Distributor, Predicted: End User

Context: If we're looking at the different customer groups, the ones we're targeting are the wholesaler and the distributor, plain and simple.
Correct: Wholesaler, Distributor, Predicted: Consultant, Planner, Architect

Context: So, the customer group that we’ve got is comprised of, let me see, Consultants, naturally, then the Planners, and you can't forget the Architects, they're a big part of it.
Correct: Consultant, Planner, Architect, Predicted: Wholesaler, Distributor

Context: The various customer groups we

## Evaluate Dataset with Fine-tuned Model

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset

# Prepare new dataset with labels
def prepare_data(df):
    records = []
    for _, row in df.iterrows():
        question = row["question"]
        context = row["context"]
        intended_answer = row["intended_answer"]
        options = row["options"]
        if options is None:
            options = row["intended_answer"] #Date und Number
            if intended_answer is None:
                options = row["context"] #Text

        for option in options:
            label = int(option == intended_answer)
            records.append({"text": f"{question} {context} {option}", "label": label})

    return pd.DataFrame(records)

prepared_df = prepare_data(data)

print(prepared_df.size)
pd.set_option('display.max_colwidth', None)
prepared_df.head()

54186


,text,label
0,"Data processing consent Absolutely, I'm totally okay with data processing, so yes. Yes",1
1,"Data processing consent Absolutely, I'm totally okay with data processing, so yes. No",0
2,"Data processing consent Yeah, you have my full consent for data processing, that's a yes from me. Yes",1
3,"Data processing consent Yeah, you have my full consent for data processing, that's a yes from me. No",0
4,"Data processing consent Oh, data processing consent? Yes, you can go right ahead with that, no problem at all. Yes",1


In [7]:
# Stratified sampling
half_df,_ = train_test_split(
    prepared_df,
    train_size=0.2, #~10.000 Einträge
    stratify=prepared_df["label"],
    random_state=42,
)

# Split into training and validation data
train_df, val_df = train_test_split(
    half_df,
    train_size=0.8,  # 80% Training, 20% Validation
    stratify=half_df["label"],
    random_state=42,
)
train_df.size

8668

In [8]:
# Tokenize data
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"].tolist(), padding="max_length", truncation=True, return_tensors="pt")

train_encodings = tokenize_function(train_df)
val_encodings = tokenize_function(val_df)

# Extract labels
train_labels = train_df["label"].values
val_labels = val_df["label"].values

# Create custom dataset
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
# Initialize model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    fp16=True,
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Erhöht
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,  # Optional: Für größere effektive Batches
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,  # Weniger Logs schreiben
    save_steps=2000,  # Modelle seltener speichern
    save_total_limit=2,
    load_best_model_at_end=True,
    warmup_steps=500,  # Optional: Lernrate langsam ansteigen lassen
    lr_scheduler_type="cosine",  # Optional: Lernraten-Scheduler
)


# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-308eba53d869>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Verwendetes Gerät: {device}")

Verwendetes Gerät: cpu


In [13]:
# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: annelie-friedl (annelie-friedl-universit-t-w-rzburg). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save model
from google.colab import drive
drive.mount('/content/drive')
!cp -r ./results /content/drive/MyDrive


# Evaluate Dataset with Finte-tuned Model

In [ ]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Pfad zu deinem trainierten Modell
model_path = "./results"

# Modell und Tokenizer laden
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


# Erstelle eine Pipeline mit dem trainierten Modell
qa_pipeline_ft = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
accuracy = predict_answers(data, qa_pipeline_ft)
print(f"Accuracy of fine-tuned model: {accuracy * 100:.2f} %")

## Evaluate continuous text

## Interesting Findings

*   Prediction of names very bad, because no deeper meaning --> fixed by checking for exact matches
  * Maybe implement name interpreter later?
*   Numerical values (size of company) prediction very bad

* QA Pipelines
  * Pipeline 2 und 3 haben nur eine accuracy von ungefähr 60 %

